# Open 2020: Winner Predictor
## Diego Chinellato - 867637
## Giorgia Campardo - 867928
### Web Intelligence Course

In [5]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import tree

In [6]:
df = pd.read_excel('2019.xlsx')
df_ext = pd.read_excel('2019+.xlsx')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 36 columns):
ATP           2234 non-null int64
Location      2234 non-null object
Tournament    2234 non-null object
Date          2234 non-null datetime64[ns]
Series        2234 non-null object
Court         2234 non-null object
Surface       2234 non-null object
Round         2234 non-null object
Best of       2234 non-null int64
Winner        2234 non-null object
Loser         2234 non-null object
WRank         2230 non-null float64
LRank         2221 non-null float64
WPts          2231 non-null float64
LPts          2221 non-null float64
W1            2215 non-null float64
L1            2215 non-null float64
W2            2205 non-null float64
L2            2205 non-null float64
W3            1119 non-null float64
L3            1119 non-null float64
W4            265 non-null float64
L4            265 non-null float64
W5            96 non-null float64
L5            96 non-null float64
Wset

In [8]:
df_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 42 columns):
ATP           23634 non-null int64
Location      23634 non-null object
Tournament    23634 non-null object
Date          23634 non-null datetime64[ns]
Series        23634 non-null object
Court         23634 non-null object
Surface       23634 non-null object
Round         23634 non-null object
Best of       23634 non-null int64
Winner        23634 non-null object
Loser         23634 non-null object
WRank         23624 non-null float64
LRank         23586 non-null float64
WPts          23626 non-null float64
LPts          23587 non-null float64
W1            23483 non-null float64
L1            23485 non-null float64
W2            23260 non-null float64
L2            23260 non-null float64
W3            11173 non-null float64
L3            11173 non-null float64
W4            2200 non-null float64
L4            2200 non-null float64
W5            816 non-null float64
L5          

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 36 columns):
ATP           2234 non-null int64
Location      2234 non-null object
Tournament    2234 non-null object
Date          2234 non-null datetime64[ns]
Series        2234 non-null object
Court         2234 non-null object
Surface       2234 non-null object
Round         2234 non-null object
Best of       2234 non-null int64
Winner        2234 non-null object
Loser         2234 non-null object
WRank         2230 non-null float64
LRank         2221 non-null float64
WPts          2231 non-null float64
LPts          2221 non-null float64
W1            2215 non-null float64
L1            2215 non-null float64
W2            2205 non-null float64
L2            2205 non-null float64
W3            1119 non-null float64
L3            1119 non-null float64
W4            265 non-null float64
L4            265 non-null float64
W5            96 non-null float64
L5            96 non-null float64
Wset

In [10]:
df.describe()

,ATP,Best of,WRank,LRank,WPts,LPts,W1,L1,W2,L2,...,Wsets,Lsets,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
count,2234.000000,2234.000000,2230.000000,2221.000000,2231.000000,2221.000000,2215.000000,2215.000000,2205.000000,2205.000000,...,2215.000000,2215.000000,2225.000000,2225.000000,2225.000000,2225.000000,2234.000000,2234.000000,2234.000000,2234.000000
mean,28.635184,3.453894,57.948430,79.944169,1723.479606,1104.742008,5.823928,4.190971,5.822676,4.043537,...,2.200451,0.445147,1.850387,3.222966,1.937683,3.526445,2.002936,3.743693,1.871629,3.186124
std,15.505220,0.837904,54.876823,76.409609,2022.360691,1145.908936,1.199468,1.828982,1.229490,1.840488,...,0.455321,0.577775,0.917769,3.228215,1.051800,3.723625,1.125870,4.409477,0.933585,2.769056
min,1.000000,3.000000,1.000000,1.000000,17.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.002000,1.070000,1.005000,1.070000,1.010000,1.080000,1.010000,1.060000
25%,17.000000,3.000000,20.000000,36.000000,707.000000,577.000000,6.000000,3.000000,6.000000,3.000000,...,2.000000,0.000000,1.300000,1.660000,1.330000,1.760000,1.360000,1.820000,1.310000,1.720000
50%,29.000000,3.000000,48.000000,64.000000,983.000000,830.000000,6.000000,4.000000,6.000000,4.000000,...,2.000000,0.000000,1.570000,2.300000,1.640000,2.440000,1.670000,2.510000,1.600000,2.340000
75%,41.000000,3.000000,78.000000,98.000000,1740.000000,1205.000000,6.000000,6.000000,6.000000,6.000000,...,2.000000,1.000000,2.100000,3.500000,2.190000,3.650000,2.270000,3.800000,2.120000,3.477500
max,54.000000,5.000000,455.000000,1415.000000,12415.000000,12355.000000,7.000000,7.000000,7.000000,7.000000,...,3.000000,2.000000,9.000000,41.000000,11.730000,37.800000,12.220000,67.000000,9.640000,28.490000


In [11]:
def compute_elo_rankings(data):
    """
    Given the list on matches in chronological order, for each match, computes 
    the elo ranking of the 2 players at the beginning of the match
    """
    print("Elo rankings computing...")
    players=list(pd.Series(list(data.Winner)+list(data.Loser)).value_counts().index)
    elo=pd.Series(np.ones(len(players))*1500,index=players)
    ranking_elo=[(1500,1500)]
    for i in range(1,len(data)):
        w=data.iloc[i-1,:].Winner
        l=data.iloc[i-1,:].Loser
        elow=elo[w]
        elol=elo[l]
        pwin=1 / (1 + 10 ** ((elol - elow) / 400))    
        K_win=32
        K_los=32
        new_elow=elow+K_win*(1-pwin)
        new_elol=elol-K_los*(1-pwin)
        elo[w]=new_elow
        elo[l]=new_elol
        ranking_elo.append((elo[data.iloc[i,:].Winner],elo[data.iloc[i,:].Loser])) 
    ranking_elo=pd.DataFrame(ranking_elo,columns=["elo_winner","elo_loser"])    
    ranking_elo["proba_elo"]=1 / (1 + 10 ** ((ranking_elo["elo_loser"] - ranking_elo["elo_winner"]) / 400))   
    return ranking_elo

In [12]:
def preprocess_data(df, 
                    features_to_drop=[], 
                    missing_values="drop", 
                    drop_first=False):
    """
    Processes raw data and returns a tuple (X, Y) where X is the cleaned dataset and Y is the array of labels.
    """
    # Sort by date to calculate ELO
    X = df.sort_values(by='Date')
    
    # Drop unuseful columns
    features_to_drop += ['ATP', 'Location', 'Tournament', 'Date', 'Comment', 
                         'WPts', 'LPts', 'Wsets', 'Lsets', 
                         'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 
                         'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW', 'PSL', 'SJW', 'SJL']
    X = X.drop(columns=features_to_drop)
    
    # Deal with missing values
    X['WRank'] = X['WRank'].fillna(value=X['WRank'].max()+1).astype(int)
    X['LRank'] = X['LRank'].fillna(value=X['LRank'].max()+1).astype(int)
    
    if missing_values == 'drop':
        X = X.dropna()
    elif missing_values == 'mean':
        from sklearn.impute import SimpleImputer
        pass
    elif missing_values == 'custom':
        pass
    else:
        raise ValueError('Wrong parameter: missing_values')

    # Convert ordinal features to int (higher value means more important)
    series = ['ATP250', 'ATP500', 'Masters 1000', 'Masters Cup', 'Grand Slam']
    series2int = {s: i for i, s in enumerate(series)}
    rounds2int = {'1st Round': 0,
                  '2nd Round': 1,
                  '3rd Round': 2,
                  '4th Round': 3,
                  'Round Robin': 4,
                  'Quarterfinals': 5,
                  'Semifinals': 6,
                  'The Final': 7,
                 }
    X = X.replace({'Round': rounds2int, 'Series': series2int})
    
    # Convert court to binary
    X = X.replace({'Court': {'Outdoor': 0, 'Indoor': 1}})
    
    # One hot encode categorical features into binary features
    X = pd.get_dummies(X, prefix=['Surface_'], columns=['Surface'], drop_first=drop_first)
    
    # Convert players to numeric ?
    players = set(X['Winner']) | set(X['Loser'])
    players_to_id = {}
    for i, player in enumerate(players):
        players_to_id[player] = i
    X = X.replace({'Winner': players_to_id, 'Loser': players_to_id})
    X = X.astype({'Winner':int, 'Loser':int})

    X = X.rename(columns={'Winner':'1st Player', 'Loser':'2nd Player', 
                          'WRank':'P1Rank', 'LRank':'P2Rank', 
                          'MaxW':'MaxP1', 'MaxL':'MaxP2', 
                          'AvgW':'AvgP1', 'AvgL':'AvgP2'})
    
    # Generate labels
    Y = np.concatenate([np.ones(X.shape[0], dtype=int), np.zeros(X.shape[0], dtype=int)])
    # Swap columns and concatenate to data
    tmp = X.copy()
    cols_to_swap = ['1st Player', '2nd Player', 'P1Rank', 'P2Rank', 'MaxP1', 'MaxP2',  'AvgP1',  'AvgP2']
    cols_swapped = ['2nd Player', '1st Player', 'P2Rank', 'P1Rank', 'MaxP2', 'MaxP1',  'AvgP2',  'AvgP1']
    tmp[cols_to_swap] = tmp[cols_swapped]
    tmp.index = np.array(range(X.shape[0] + 1, X.shape[0] * 2 + 1))
    X = pd.concat((X, tmp))
    
    return X, Y

In [13]:
data, labels = preprocess_data(df_ext)

In [14]:
print(all(type(t) == np.int32 for t in labels))

True


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47218 entries, 220 to 47218
Data columns (total 15 columns):
Series            47218 non-null int64
Court             47218 non-null int64
Round             47218 non-null int64
Best of           47218 non-null int64
1st Player        47218 non-null int32
2nd Player        47218 non-null int32
P1Rank            47218 non-null int32
P2Rank            47218 non-null int32
MaxP1             47218 non-null float64
MaxP2             47218 non-null float64
AvgP1             47218 non-null float64
AvgP2             47218 non-null float64
Surface__Clay     47218 non-null uint8
Surface__Grass    47218 non-null uint8
Surface__Hard     47218 non-null uint8
dtypes: float64(4), int32(4), int64(4), uint8(3)
memory usage: 4.1 MB


In [21]:
def foo(X, Y):
    # Division of dataset in training set and test set.
    # Validation set not required since we're going to use cross-validation
    X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)
    # do we need to preserve date ordering?
    # test with decision trees
    print("{:10} - {:10} - {:10} - {:10}".format('Leaves', 'Train acc', 'Test acc', 'k-valid'))
    for leaves in range(2, 50):
        dt = tree.DecisionTreeClassifier(class_weight="balanced",
                                         max_leaf_nodes=leaves)
        # 1st method, no validation set
        # not sure if it's right the way it's done
        dt.fit(X_train, Y_train)
        train_acc = accuracy_score(y_true=Y_train, y_pred=dt.predict(X_train))
        test_acc = accuracy_score(y_true=Y_test, y_pred=dt.predict(X_test))
        
        # 2nd method using cross validation, should perform better
        dt = tree.DecisionTreeClassifier(class_weight="balanced",
                                         max_leaf_nodes=leaves)
        scores = cross_val_score(dt, X_train, Y_train, 
                                 cv=5, scoring='accuracy')
        
        print(leaves, train_acc,test_acc, scores.mean())

In [22]:
foo(data, labels)

Leaves     - Train acc  - Test acc   - k-valid   
2 0.7000254143958433 0.7025836509953409 0.6993760511732772
3 0.7000254143958433 0.7025836509953409 0.6993760511732772
4 0.7000254143958433 0.7025836509953409 0.6993760511732772
5 0.7000254143958433 0.7025836509953409 0.6993760511732772
6 0.7000254143958433 0.7025836509953409 0.6993760511732772
7 0.7000254143958433 0.7025836509953409 0.6993760511732772
8 0.7000254143958433 0.7025836509953409 0.6993760511732772


KeyboardInterrupt: 

In [ ]:
def dowload_ao2020_data():
    """
    To get the data for the tournament? if we need to scrape these are the links of the matches
    https://ausopen.com/tournament-schedule
    https://ausopen.com/schedule#!8071
    """
    
def get_elo():
    from requests import get
    